# Cadence Effects

This notebook uses simulates normal Type Ia Supernova (SN Ia) light-curves using realistic cadences and atmospheric variabilities expected from LSST. To achieve this we proceed as follows:

1. Impliment an interpolator that determines the desired PWV as a function of time (MJD).
2. Use data from the PLaSTICC simulations to establish the cadence, light-curve parameters, and location of SNe observed by LSST.
3. Apply time variable PWV transmission effects to a simulated light-curve
4. Simulate and fit a handful of light-curves for a single cadence and anlyize the results.



In [ ]:
import sys
import warnings
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
import sncosmo
from astropy.io import fits
from astropy.table import Table
from astropy.time import Time
from matplotlib import pyplot as plt
from pwv_kpno.gps_pwv import GPSReceiver

sys.path.insert(0, '../')
from sn_analysis import filters, plasticc, plotting, sn_magnitudes, modeling 


In [ ]:
filters.register_lsst_filters(force=True)

plt.rcParams['figure.dpi'] = 100
print('Simulation data directory:', plasticc.plasticc_simulations_directory)


## Atmospheric Variability

To create a physically reasonable representation of the atmospheric variability at LSST, we use PWV measurements taken at the nearby Cerro Telolo International Observatory (CTIO).


In [ ]:
ctio = GPSReceiver('CTIO', data_cuts={'PWV': [(0, 25)]})
ctio.download_available_data(year=range(2012, 2018))


In [ ]:
ctio_weather = ctio.weather_data()

ctio_weather.reset_index().plot.scatter('date', 'PWV', s=1, figsize=(10, 4), alpha=.2)
plt.ylabel('CTIO PWV (mm)')
plt.xlabel('Date')
plt.title('All available PWV measurements for CTIO')
plt.ylim(0, 25)


We don't have enough data to fully represent a 10 year long survey. Fortunately we are mostly interested in timescales of seasonal variability and shorter so we can consider data from a single year with good measurement coverage. The two years with the best coverage are 2016 and 2017.

In [ ]:
for year in range(2012, 2018):
    weather_for_year = ctio_weather[ctio_weather.index.year == year]
    plotting.plot_year_pwv_vs_time(weather_for_year.PWV)
    plt.title(f'CTIO PWV over {year}');
    plt.show()


Instead of thinking through some fancy statistics, we keep our approach simple and suppliment missing data from 2016 with measurements taken in 2017.


In [ ]:
stacked_pwv = ctio_weather[np.isin(ctio_weather.index.year, (2016, 2017))].PWV
stacked_pwv = stacked_pwv.sort_index()

new_index = []
for date_idx in stacked_pwv.index:
    new_index.append(date_idx.replace(year=2016))
    
# Suppliment missing data
stacked_pwv.index = new_index
stacked_pwv = stacked_pwv[~stacked_pwv.index.duplicated(keep='first')]

# Resample and interpolate any missing values
stacked_pwv = stacked_pwv.resample('30min', offset=timedelta(minutes=15)).interpolate()

plotting.plot_year_pwv_vs_time(stacked_pwv)
plt.title('PWV Model');


There may stil a few periods of missing data in the above figure. Fortunatly these periods are short so they can be interpolated over. To interpolate the PWV for a given datetime, we reindex our PWV model using the number of seconds that have elapsed in a given year. To avoid extrapolating at the boundaries, we extend our PWV model to slightly before / after the beggining / end of the year by wrapping the data across the boundaries.


In [ ]:
def datetime_to_sec_in_year(dates):
    """Calculate number of seconds elapsed modulo 1 year
    
    Args:
        dates (pd.Datetime): Pandas datetime array
        
    Returns:
        A numpy array of integers
    """
    
    dates = pd.to_datetime(dates)
    
    hour_in_day = 24
    min_in_hour = sec_in_min = 60
    return (
        dates.dayofyear * hour_in_day * min_in_hour * sec_in_min
        + dates.hour * min_in_hour * sec_in_min
        + dates.minute * sec_in_min
    )


In [ ]:
# Convert index values to seconds
pwv_model = stacked_pwv.copy()
pwv_model.index = datetime_to_sec_in_year(pwv_model.index)

# Resample the index to span the whole year
end_of_year = 365.25 * 24 * 60 * 60  # Days in year * hours * min * sec
delta = pwv_model.index[1] - pwv_model.index[0]
offset = pwv_model.index[1] % delta
new_indices = np.arange(-offset, end_of_year + offset + 2 * delta, delta)
pwv_model = pwv_model.reindex(new_indices) 

# Wrap values across the boundaries and fill nans with interpolation
first_not_nan, *_, last_not_nan = np.where(~pwv_model.isna())[0]
pwv_model.iloc[0] = pwv_model.iloc[last_not_nan]
pwv_model.iloc[-1] = pwv_model.iloc[first_not_nan]
pwv_model = pwv_model.interpolate()


In [ ]:
pwv_model.plot(marker='o', linestyle='', markersize=1)
plt.xlabel('Seconds in year')
plt.ylabel('PWV (mm)')
plt.title('PWV Model')
plt.xlim(pwv_model.index.min(), pwv_model.index.max())
plt.ylim(0, 25)


In [ ]:
def build_interpolater_from_suomi_data(pwv_model):
    """Build interpolator for the PWV at a given point of the year
    
    Args:
        pwv_model (Series): PWV values indexed by seconds in year
        
    Returns:
        An interpolation function that accepts MJD
    """
    
    def interp(mjd):
        
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            x_as_datetime = Time(mjd, format='mjd').to_datetime()
            x_in_seconds = datetime_to_sec_in_year(x_as_datetime)
                
        return np.interp(
            x=x_in_seconds,
            xp=pwv_model.index,
            fp=pwv_model.values
        )
    
    return interp


In [ ]:
pwv_interpolator = build_interpolater_from_suomi_data(pwv_model)


We make a quick validation plot to check the interpolation function.


In [ ]:
def plot_interpolation_validation(pwv_data, pwv_interpolator):
    """Overplot the interpolated and measured PWV
    
    Args:
        df (DataFrame): Dataframe with PWV values and a Datetime index
        year   (float): Year of data to use from ``df``
    """
    
    pwv_data = pwv_data.sort_index()
    interpolated_pwv = pwv_interpolator(Time(pwv_data.index).mjd)

    plt.figure(figsize=(9, 3))
    plt.scatter(pwv_data.index, pwv_data.values, s=2, alpha=.1, label='Measured')
    plt.plot(pwv_data.index, interpolated_pwv, label='Interpolated', color='C1', alpha=.75)
    plt.ylabel('PWV (mm)')
    plt.xlabel('Date (UTC)')
    plt.legend()


In [ ]:
plot_interpolation_validation(ctio_weather.PWV, pwv_interpolator)
plt.xlim(datetime(2016, 1, 1), datetime(2018, 1, 1))


## The PLaSTICC Data

Instead of evaluating different cadences from scratch, we use light-curves from the PLaSTICC simulations. First we check what cadence simulations are available on the notebook's host server.


In [ ]:
plasticc.get_available_cadences()


Simulated light-curves are written in the SNANA file format and are distributed across multiple files. We load a light-curve from one of these files and demosntrate the data model below. Each cadence includes simulations run with multiple supernova models. In this notebook we only need simulations for normal SNe (Model 11). 


In [ ]:
demo_cadence = 'alt_sched'
demo_cadence_header_files = plasticc.get_model_headers(demo_cadence, 11)

print('Available cadence files:', len(demo_cadence_header_files))
print('Available Light-curves: ', plasticc.count_light_curves(demo_cadence, model=11))
    

In [ ]:
demo_header_path = demo_cadence_header_files[0]
plasticc_lc = next(plasticc.iter_lc_for_header(demo_header_path, verbose=False))


In [ ]:
plasticc_lc.meta


In [ ]:
plasticc_lc


Here we reformat the data to be compatible with `sncosmo` so we can easily visualize the light-curve.


In [ ]:
formatted_lc = plasticc.format_plasticc_sncosmo(plasticc_lc)


In [ ]:
sncosmo.plot_lc(formatted_lc);


## Simulating Light-Curves

Since we need to add in our own atmospheric variability, the pre-tabulated flux values above are of limited use. Instead, we use the PLaSTICC light-curves to establish the cadence and model parameters for each simulated SN. This information is then used to simulate our own light-curves with `sncosmo`.


**Note:** See Issue 8 (https://github.com/LSSTDESC/SN-PWV/issues/8) for caveats about the following cell.

In [ ]:
model_for_sim = sncosmo.Model('salt2-extended')
duplicated_lc = plasticc.duplicate_plasticc_sncosmo(plasticc_lc, model_for_sim, gain=20, skynr=100)


In [ ]:
duplicated_lc.meta


In [ ]:
sncosmo.plot_lc(duplicated_lc);


In [ ]:
duplicated_lc


The `sncosmo` package doesn't have a clearly defined approach to adding time variabile propagation effects, so we use our own custom modeling class. The `PWVVariableModel` class is a child of the `sncosmo.Model` class and overloads the underlying flux calculation by adding PWV transmission effects. Note that in this approach the `t0` parameter for each light-curve is now in units of MJD (i.e., it must be the same units as the interpolation function).


In [ ]:
def plot_variable_pwv_model(model_with_pwv, phase=0, params=dict()):
    """Overplot a sncosmo model with and without temporally variable PWV
    
    Args:
        source (str, Source): sncosmo source to plot
        phase        (float): Phase of the supernova to plot
        params        (dict): Non-PWV related parameters for the model
    """
    
    wave = np.arange(3000, 12000)
    time = phase + params['t0']

    model_without_pwv = sncosmo.Model(model_with_pwv.source)
    model_without_pwv.update(params)
    flux_without_pwv = model_without_pwv.flux(time, wave)

    model_with_pwv.update(params)
    flux_with_pwv = model_with_pwv.flux(time, wave)

    fig, ax = plt.subplots(1, 1, figsize=(6, 3))
    ax.plot(wave, flux_without_pwv, label='Base Model', color='C1')
    ax.plot(wave, flux_with_pwv, label='Model with PWV', color='C0')
    ax.set_title('Simulated Flux')
    ax.set_ylabel('Flux')
    ax.legend()
    ax.set_xlabel('Wavelength (A)')
    ax.set_xlim(min(wave), max(wave))

    plt.tight_layout()


In [ ]:
demo_model = modeling.Model(
    source='salt2-extended',
    effects=[modeling.VariablePWVTrans(pwv_interpolator)],
    effect_names=['PWV_transmission'],
    effect_frames=['obs']
)

plot_variable_pwv_model(
    demo_model,     
    params = {
        'z': 0.752069652,
        't0': 61900,
        'x0': 3e-06,
        'x1': -1.8,
        'c': -0.1}
)


## Fitting Light-Curves

We create an iterator that extracts cadence data from PLaSTICC light-curves and simulates custom light-curves with time variable PWV effects. We then fit each light-curve and look at the aggregate properties. In order to ensure a successful fit for each light-curve, we apply the following quality cuts:

1. Light-curves must have at least one point with SNR >= 5 in two or more bands
2. So that the simulated data is within the wavelength range of the model, light-curves with a redshift greater than .8 are dropped. 

In [ ]:
def iter_custom_lcs(
        model, cadence, iter_lim=None, gain=20, skynr=100, quality_callback=None, verbose=True):
    """Simulate light-curves for a given PLaSTICC cadence
    
    Args:
        model               (Model): Model to use in the simulations
        cadence               (str): Cadence to use when simulating light-curves
        gain                  (int): Gain to use during simulation
        skynr                 (int): Simulate skynoise by scaling plasticc ``SKY_SIG`` by 1 / skynr
        quality_callback (callable): Skip light-curves if this function returns False
        verbose              (bool): Display a progress bar
    """
    
    u_band_low = sncosmo.get_bandpass('lsst_hardware_u').minwave()
    source_low = model.source.minwave()
    zlim = (u_band_low / source_low) - 1
    
    counter = -1
    iter_lim = float('inf') if iter_lim is None else iter_lim
    for light_curve in plasticc.iter_lc_for_cadence_model(cadence, model=11, verbose=verbose):
        counter += 1
        if counter >= iter_lim:
            break
        
        if light_curve.meta['SIM_REDSHIFT_CMB'] >= zlim:
            continue
        
        duplicated_lc = plasticc.duplicate_plasticc_sncosmo(light_curve, model, gain=gain, skynr=skynr)
        if quality_callback and not quality_callback(duplicated_lc):
            continue
            

        yield duplicated_lc 
        

In [ ]:
def passes_quality_cuts(light_curve):
    """Return whether light-curve has 2+ two bands each with 1+ data point with SNR > 5
    
    Args:
        light_curve (Table): Astropy table with sncosmo formatted light-curve data
        
    Returns:
        A boolean
    """
    
    if light_curve.meta['z'] > .88:
        return False
    
    light_curve = light_curve.group_by('band')
    
    passed_cuts = []
    for band_lc in light_curve.groups:
        passed_cuts.append((band_lc['flux'] /  band_lc['fluxerr'] > 5).any())
        
    return sum(passed_cuts) >= 2
        

We pause to visually check a light-curves from our iterator. As a simple validation, we simulate light-curves with and without PWV.

In [ ]:
l = next(iter_custom_lcs(demo_model, demo_cadence, verbose=False))
sncosmo.plot_lc(l);


In [ ]:
model_without_pwv = sncosmo.Model('salt2-extended')
l = next(iter_custom_lcs(model_without_pwv, demo_cadence, verbose=False))
sncosmo.plot_lc(l);


In [ ]:
light_curves = iter_custom_lcs(demo_model, demo_cadence, iter_lim=10, quality_callback=passes_quality_cuts)

model_without_pwv = sncosmo.Model('salt2-extended')
fitted_mag, fitted_params = sn_magnitudes.fit_mag(
    model=model_without_pwv, 
    light_curves=light_curves, 
    vparams=['x0', 'x1', 'c'], 
    bands=['lsst_hardware_' + b for b in 'ugrizy'])
